In [ ]:
!pip install pysmb

In [52]:
import platform
from smb.SMBConnection import SMBConnection

import pandas as pd
import datetime as dt
import io
import re
import os
import json

In [111]:
# connection open
ip_address = '172.30.9.85'
user = 'Administrator'
password = 'panic'
remote_hostname = 'RGSV920'

#ip_address = '172.30.4.193'
#user = 'Administrator'
#password = 'panic'
#remote_hostname = 'SESV400'

conn = SMBConnection(
    user,
    password,
    platform.uname().node,
    remote_hostname,
    domain='WORKGROUP',
    use_ntlm_v2=True)
conn.connect(ip_address, 139)

items = conn.listPath('d$', 'Ftp/Logs/2021')

#print(dir(items[0]))
data = []
order = [
    "date","time","c-ip","cs-username","s-ip",
    "s-port","cs-method","cs-uri-stem","sc-status","sc-win32-status",
    "sc-substatus","time-taken","x-session","x-fullpath"
]
res = pd.DataFrame(columns = order)

for item in items:
    #print(f'%s %s %d %s %s' % 
    #      (
    #          item.filename, 
    #          item.isDirectory,
    #          item.alloc_size,
    #          dt.datetime.fromtimestamp(item.last_access_time),
    #          dt.datetime.fromtimestamp(item.last_write_time)
    #      )
    #     )
    if(not item.isDirectory and re.search('210101', item.filename)):
        print(f'%s %d' % (item.filename, item.alloc_size)) 
        
        with open('/home/jovyan/datasets/Ftplogs/'+ item.filename, 'wb') as file:
            conn.retrieveFile('d$', 'Ftp/Logs/2021/' + item.filename, file)
            
        with io.BytesIO() as file:
            conn.retrieveFile('d$', 'Ftp/Logs/2021/' + item.filename, file)
            file.seek(0)
            lines = file.read().splitlines()
            #lines = file.readlines()
            #print(len(lines))
            
            cnt = 0
            for buf in lines:
                cnt+=1
                if not re.search('^#', buf.decode()):
                    #details = buf.decode().split(' ')
                    #details = [x.strip() for x in details]
                    #structure = {key:value for key, value in zip(order, details)}
                    #data.append(structure)
                    #cols = buf.decode().split(' ')
                    (date,time,cip,csusername,sip,
                     sport,csmethod,csuristem,scstatus,scwin32status,
                     scsubstatus,timetaken,xsession,xfullpath ) = buf.decode().split(' ')
                    res.loc[len(res)] = [
                        date,time,cip,csusername,sip,
                     sport,csmethod,csuristem,scstatus,scwin32status,
                     scsubstatus,timetaken,xsession,xfullpath]
                    #DATE, TIME, IP1, FIL1, iP2, PORT, METHOD, FIL2, FIL3, ST1, ST2, ST3,ST4, FIL4 = buf.decode().split(' ')
                    #res.loc[len(res)]=[DATE, TIME, IP1, FIL1, iP2, PORT, METHOD, FIL2, FIL3, ST1, ST2, ST3,ST4, FIL4]
                    #print(cols)
                    #print(f'[%5d]%s' % (cnt, buf.decode()))
            
            file.close()
            
            #for entry in data:
            #    print(json.dumps(entry, indent = 4))
            
df_concat = pd.DataFrame()
df_concat = pd.concat([df_concat, res])
conn.close()

u_ex210101.log 20480


In [110]:
#print(df_concat.iloc[:,0].isna().sum())

for col in df_concat.columns:
    print(f'%s [%s]\n\t%s' % (col, df_concat.iloc[:,0].isna().sum(), df_concat.loc[:,col].unique()))

date [0]
	['2021-01-01']
time [0]
	['10:30:31' '10:30:33']
c-ip [0]
	['172.31.74.122']
cs-username [0]
	['-' 'RGSV920\\HOST0001']
s-ip [0]
	['172.30.9.85']
s-port [0]
	['21' '51094' '51095' '51096' '51097' '51098' '51099' '51100' '51101'
 '51102' '51103' '51104' '51105']
cs-method [0]
	['ControlChannelOpened' 'SYST' 'USER' 'PASS' 'TYPE' 'CWD' 'PASV'
 'DataChannelOpened' 'DataChannelClosed' 'STOR' 'QUIT'
 'ControlChannelClosed']
cs-uri-stem [0]
	['-' 'host0001' '***' 'A' '/ftp6001/Send/ServerLink' 'SS-KZEND-KOZU.TXT'
 'SS-KZEND-BUZAI.TXT' 'SS-KZEND-BUHIN.TXT' 'SS-KZEND-KAKO-BUHIN.TXT'
 'SS-KZEND-KOSAKU-SHIJI.TXT' 'SS-KZEND.ZCSV2' 'SS-KZEND-KNP-KOZU.TXT'
 'SS-KZEND-KNP-BUZAI.TXT' 'SS-KZEND-KNP-BUHIN.TXT'
 'SS-KZEND-KNP-KAKO-BUHIN.TXT' 'SS-KZEND-KNP-KOSAKU-SHIJI.TXT'
 'SS-KZEND-KNP.ZCSV2']
sc-status [0]
	['-' '215' '331' '230' '200' '250' '227' '226' '221']
sc-win32-status [0]
	['0']
sc-substatus [0]
	['0']
time-taken [0]
	['0']
x-session [0]
	['0595f56c-280e-4080-af4e-d1b237774105'
 '962